In [6]:
import os
current_path = os.getcwd()
print(current_path)
os.chdir("..")
current_path = os.getcwd()
print(current_path)

/home/xzhang/Documents/我的模型
/home/xzhang/Documents


In [7]:
# 第一个实验用于解决learning curve不平滑的问题
# learning curve 有很多peak， 我们要尝试多种可能解决掉这个问题 tune.run
# trial 1 改变学习率，找到最优学习率
# 配置： 迭代了10000次，输入uniform 0 0.1, target ground_truth
# 观察到大的学习率会出现震荡，小的学习率效果不错，数量级大约在0.0007，0.0004，0.0001，这样数量级的学习率不会出现问题，
# 结果保存在test1_logs中了
# 接下来 trial 2
# 试着跑1000次看看效果，并且测试不同的targets，看看有什么影响，也在test1_logs中
# 模型相关
import torch
from models.DIP_2D import DIP_2D
import pytorch_lightning as pl
import numpy as np

# 画图相关
import matplotlib.pyplot as plt

# 文件读写相关
import csv
from config.config_tune import *
from config.config import config
import pandas as pd
import csv

# 自定义函数
from utils.pre_utils import *

from functools import partial

import datetime




In [8]:
path_input = "data/noisy_images/uniform_noise.npy"
path_ground_truth = "data/ground_truth/ground_truth.npy"
path_target_list = ["data/ground_truth/ground_truth.npy","data/corrupted_images/BSREM_it30.npy","data/noisy_images/uniform_noise2.npy"]

In [9]:
image_net_input = np.load(path_input)
# image_net_input_scale,param1_scale_im_net,param2_scale_im_net = rescale_imag(image_net_input,"standardization") 
# image_net_input_torch = torch.Tensor(image_net_input_scale)
image_net_input_torch = torch.Tensor(image_net_input)
image_net_input_torch = image_net_input_torch.view(1,1,PETImage_shape[0],PETImage_shape[1],PETImage_shape[2])
image_net_input_torch = image_net_input_torch[:,:,:,:,0]
ground_truth = np.load(path_ground_truth)

def main(config,train_dataloader,param1,param2,image_corrupt,target):
    checkpoint_simple_path = os.getcwd()
    # 创建TensorBoardLogger，注意savedir和后面打开tensorboard时的路径要一致，versions是啥？
    logger = pl.loggers.TensorBoardLogger(save_dir=checkpoint_simple_path)
    # 读取并导入模型
    model = DIP_2D(param1, param2,config,'data/Algo/',
                "nested",all_images_DIP="Last",global_it=-100, suffix="suffix", last_iter=-1,ground_truth=ground_truth)
    model_class = DIP_2D
    # 设置log路径保存在runs目录

    print(config["sub_iter_DIP"])
    trainer = pl.Trainer(max_epochs=config["sub_iter_DIP"],log_every_n_steps=100,logger=logger)#, callbacks=[checkpoint_callback, tuning_callback, early_stopping_callback], logger=logger,gpus=gpus, accelerator=accelerator, profiler="simple")
    trainer.fit(model, train_dataloader)
    out = model(image_net_input_torch)
    image_out = out.view(PETImage_shape[0],PETImage_shape[1],PETImage_shape[2]).detach().numpy()
    image_concat = np.concatenate((image_corrupt, destand_numpy_imag(image_out,param1,param2)), axis=1)
    image_reversed =np.max(image_concat)-image_concat
    plt.imsave(f"{config_tune['lr']}_{target.split('/')[-1].split('.')[0]}.png", np.squeezeimage_reversed, cmap='gray')
    # plt.imshow(image_reversed, cmap='gray')
    # plt.show()  

FileNotFoundError: [Errno 2] No such file or directory: 'data/noisy_images/uniform_noise.npy'

In [ ]:

for path_target in path_target_list:

    image_corrupt = np.load(path_target) # 读取图片并将图片转换成numpy array
    image_corrupt_input_scaled,param1_scale_im_corrupt,param2_scale_im_corrupt = rescale_imag(image_corrupt,"standardization") # 标准化图片, 减去平均值，除以标准差，参数1是mean，参数2是std
    image_corrupt_torch = torch.Tensor(image_corrupt_input_scaled)
    image_corrupt_torch = image_corrupt_torch.view(1,1,PETImage_shape[0],PETImage_shape[1],PETImage_shape[2])
    image_corrupt_torch = image_corrupt_torch[:,:,:,:,0]

    # 用dataset和dataloader 读取训练数据 input和目标
    train_dataset = torch.utils.data.TensorDataset(image_net_input_torch, image_corrupt_torch)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1) 

    tune.run(partial(main,train_dataloader=train_dataloader,param1=param1_scale_im_corrupt,param2=param2_scale_im_corrupt,image_corrupt=image_corrupt,target=path_target), 
             config=config_tune,local_dir = 'test1_logs')

staaaaaaaaaaand


2023-04-25 16:36:18,312	INFO worker.py:1553 -- Started a local Ray instance.


(pid=1005202) /home/xzhang/miniconda3/envs/dip/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
(pid=1005202)   warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
2023-04-25 16:36:22,043	ERROR trial_runner.py:1062 -- Trial main_8ae67_00000: Error processing event.
ray.exceptions.RayTaskError(TypeError): ray::ImplicitFunc.train() (pid=1005202, ip=172.27.198.43, repr=func)
  File "/home/xzhang/miniconda3/envs/dip/lib/python3.8/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/home/xzhang/miniconda3/envs/dip/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py", line 337, in entrypoint
    return self._trainable_func(
  File "/home/xzhang/miniconda3/envs/dip/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py", line 

Trial name,date,experiment_id,hostname,node_ip,pid,timestamp,trial_id
main_8ae67_00000,2023-04-25_16-36-21,f4a13d938c314117abe465872ae6bfe0,pc-chu-208403,172.27.198.43,1005202,1682433381,8ae67_00000
main_8ae67_00004,2023-04-25_16-36-24,ca1f65daaf3e4b31b4a7e54dc9889e82,pc-chu-208403,172.27.198.43,1005329,1682433384,8ae67_00004
main_8ae67_00005,2023-04-25_16-36-24,9aff26e2bf0c414aaf7d236b48916b22,pc-chu-208403,172.27.198.43,1005331,1682433384,8ae67_00005


(pid=1005333) /home/xzhang/miniconda3/envs/dip/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
(pid=1005333)   warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
(pid=1005327) /home/xzhang/miniconda3/envs/dip/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
(pid=1005327)   warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
(pid=1005329) /home/xzhang/miniconda3/envs/dip/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
(pid=1005329)   warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
(

TuneError: ('Trials did not complete', [main_8ae67_00000, main_8ae67_00001, main_8ae67_00002, main_8ae67_00003, main_8ae67_00004, main_8ae67_00005, main_8ae67_00006, main_8ae67_00007])